In [195]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

from sklearn.impute import KNNImputer

from scipy.stats import shapiro
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.preprocessing import OneHotEncoder

from sklearn.inspection import permutation_importance
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV

import pickle

In [196]:
immoSV_df = pd.read_csv(r'C:\Users\Admin\Documents\marianneSimplon\simplon\immo_SiliconValley_marianneD\data\traindata_ori.csv',delimiter=',', decimal='.')
immoSV_df

,Unnamed: 0,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,2072,-119.84,36.77,6.0,1853.0,473.0,1397.0,417.0,1.4817,72000.0,INLAND
1,10600,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,<1H OCEAN
2,2494,-120.19,36.60,25.0,875.0,214.0,931.0,214.0,1.5536,58300.0,INLAND
3,4284,-118.32,34.10,31.0,622.0,229.0,597.0,227.0,1.5284,200000.0,<1H OCEAN
4,16541,-121.23,37.79,21.0,1922.0,373.0,1130.0,372.0,4.0815,117900.0,INLAND
...,...,...,...,...,...,...,...,...,...,...,...
16507,1099,-121.90,39.59,20.0,1465.0,278.0,745.0,250.0,3.0625,93800.0,INLAND
16508,18898,-122.25,38.11,49.0,2365.0,504.0,1131.0,458.0,2.6133,103100.0,NEAR BAY
16509,11798,-121.22,38.92,19.0,2531.0,461.0,1206.0,429.0,4.4958,192600.0,INLAND
16510,6637,-118.14,34.16,39.0,2776.0,840.0,2546.0,773.0,2.5750,153500.0,<1H OCEAN


In [197]:
# Supression de la colonne Unnamed: 0
immoSV_df = immoSV_df.drop("Unnamed: 0", axis=1)

In [198]:
immoSV_df.dropna(axis=0, inplace=True)

In [199]:
immoSV_df = immoSV_df.reset_index(inplace=False)
immoSV_df = immoSV_df.drop("index", axis=1)

In [200]:
df = immoSV_df.copy()

In [386]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-119.84,36.77,6.0,1853.0,473.0,1397.0,417.0,1.4817,72000.0,INLAND
1,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,<1H OCEAN
2,-120.19,36.60,25.0,875.0,214.0,931.0,214.0,1.5536,58300.0,INLAND
3,-118.32,34.10,31.0,622.0,229.0,597.0,227.0,1.5284,200000.0,<1H OCEAN
4,-121.23,37.79,21.0,1922.0,373.0,1130.0,372.0,4.0815,117900.0,INLAND
...,...,...,...,...,...,...,...,...,...,...
16331,-121.90,39.59,20.0,1465.0,278.0,745.0,250.0,3.0625,93800.0,INLAND
16332,-122.25,38.11,49.0,2365.0,504.0,1131.0,458.0,2.6133,103100.0,NEAR BAY
16333,-121.22,38.92,19.0,2531.0,461.0,1206.0,429.0,4.4958,192600.0,INLAND
16334,-118.14,34.16,39.0,2776.0,840.0,2546.0,773.0,2.5750,153500.0,<1H OCEAN


### Split du dataset en un dataset train et un dataset test

In [201]:
immoSV_train, immoSV_test = train_test_split(immoSV_df, test_size=0.3)

In [202]:
immoSV_train.shape

(11435, 10)

In [203]:
immoSV_test.shape

(4901, 10)

### Scaling par normalisation

In [204]:
# Liste des colonnes de données numériques à Scaler
immoSV_num_columns_list = list(immoSV_train.columns)
immoSV_num_columns_list.remove('ocean_proximity')
immoSV_num_columns_list.remove('median_house_value')

In [205]:
sc = MinMaxScaler()
ft_sc = sc.fit(immoSV_train[immoSV_num_columns_list])
immoSV_train[immoSV_num_columns_list] = ft_sc.transform(immoSV_train[immoSV_num_columns_list])

In [206]:
immoSV_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
1839,0.721095,0.138298,0.137255,0.025948,0.030585,0.016901,0.028952,0.159963,136300.0,INLAND
10378,0.181542,0.589362,0.313725,0.244295,0.328986,0.131198,0.338378,0.253141,209800.0,NEAR BAY
13113,0.194726,0.558511,1.000000,0.071002,0.095948,0.026654,0.093601,0.278617,500001.0,NEAR BAY
10137,0.750507,0.029787,0.254902,0.318146,0.388449,0.178985,0.383287,0.193666,168700.0,<1H OCEAN
1264,0.473631,0.200000,0.941176,0.045130,0.059308,0.035427,0.062017,0.208335,251200.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
13698,0.618661,0.145745,0.843137,0.028645,0.035864,0.020853,0.036519,0.103454,92600.0,<1H OCEAN
6385,0.614604,0.144681,0.588235,0.051235,0.095327,0.062446,0.097055,0.136150,151200.0,<1H OCEAN
2965,0.116633,0.702128,0.509804,0.024905,0.036330,0.016508,0.036519,0.103964,96200.0,<1H OCEAN
15250,0.257606,0.510638,0.549020,0.060673,0.069554,0.055971,0.078631,0.267245,185500.0,<1H OCEAN


In [207]:
immoSV_test[immoSV_num_columns_list] = ft_sc.transform(immoSV_test[immoSV_num_columns_list])

In [208]:
immoSV_test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
7902,0.216024,0.524468,0.901961,0.065100,0.120944,0.086998,0.118605,0.159025,229500.0,NEAR OCEAN
8318,0.676471,0.164894,0.196078,0.187769,0.278218,0.107570,0.266162,0.160874,163000.0,INLAND
6139,0.243408,0.504255,0.156863,0.037956,0.058531,0.019956,0.056917,0.285320,234300.0,<1H OCEAN
2230,0.596349,0.198936,0.470588,0.081712,0.080733,0.049329,0.083073,0.330258,183000.0,<1H OCEAN
9576,0.215010,0.559574,1.000000,0.036175,0.034932,0.017069,0.034545,0.294541,314700.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
8556,0.251521,0.514894,0.294118,0.033656,0.059773,0.030830,0.067774,0.221749,229800.0,<1H OCEAN
10056,0.613590,0.160638,0.901961,0.035081,0.134762,0.080075,0.135549,0.066765,137500.0,<1H OCEAN
1858,0.417850,0.564894,0.294118,0.007963,0.008850,0.003700,0.009541,0.135281,137500.0,INLAND
4284,0.647059,0.146809,0.588235,0.049963,0.057289,0.025645,0.056588,0.269707,202000.0,<1H OCEAN


In [209]:
immoSV_train_sc = immoSV_train.copy()
immoSV_test_sc = immoSV_test.copy()

In [210]:
immoSV_train_sc.shape

(11435, 10)

In [211]:
immoSV_test_sc.shape

(4901, 10)

### Encoding par OneHotEncoder

In [212]:
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = enc.fit(immoSV_train_sc[['ocean_proximity']])
enc_df = pd.DataFrame(enc.transform(immoSV_train_sc[['ocean_proximity']]).toarray())

In [213]:
enc_df.shape

(11435, 5)

In [214]:
immoSV_train_sc.shape

(11435, 10)

In [215]:
immoSV_train_sc = immoSV_train_sc.reset_index()
immoSV_train_sc = immoSV_train_sc.merge(enc_df,how='inner', on=immoSV_train_sc["index"])
immoSV_train_sc = immoSV_train_sc.drop(['key_0','index','ocean_proximity'], axis=1)
immoSV_train_sc.rename({0: 'NEAR_OCEAN', 1: 'NEAR_BAY',2: 'ISLAND', 3: 'INLAND', 4: 'H_OCEAN'}, inplace=True, axis=1)
immoSV_train_sc

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,NEAR_OCEAN,NEAR_BAY,ISLAND,INLAND,H_OCEAN
0,0.721095,0.138298,0.137255,0.025948,0.030585,0.016901,0.028952,0.159963,136300.0,0.0,1.0,0.0,0.0,0.0
1,0.181542,0.589362,0.313725,0.244295,0.328986,0.131198,0.338378,0.253141,209800.0,0.0,0.0,0.0,1.0,0.0
2,0.194726,0.558511,1.000000,0.071002,0.095948,0.026654,0.093601,0.278617,500001.0,0.0,0.0,0.0,1.0,0.0
3,0.750507,0.029787,0.254902,0.318146,0.388449,0.178985,0.383287,0.193666,168700.0,1.0,0.0,0.0,0.0,0.0
4,0.473631,0.200000,0.941176,0.045130,0.059308,0.035427,0.062017,0.208335,251200.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11430,0.618661,0.145745,0.843137,0.028645,0.035864,0.020853,0.036519,0.103454,92600.0,1.0,0.0,0.0,0.0,0.0
11431,0.614604,0.144681,0.588235,0.051235,0.095327,0.062446,0.097055,0.136150,151200.0,1.0,0.0,0.0,0.0,0.0
11432,0.116633,0.702128,0.509804,0.024905,0.036330,0.016508,0.036519,0.103964,96200.0,1.0,0.0,0.0,0.0,0.0
11433,0.257606,0.510638,0.549020,0.060673,0.069554,0.055971,0.078631,0.267245,185500.0,1.0,0.0,0.0,0.0,0.0


In [216]:
enc_df = pd.DataFrame(enc.transform(immoSV_test_sc[['ocean_proximity']]).toarray())

In [217]:
enc_df.shape

(4901, 5)

In [218]:
immoSV_test_sc = immoSV_test_sc.reset_index()
immoSV_test_sc = immoSV_test_sc.merge(enc_df,how='inner', on=immoSV_test_sc["index"])
immoSV_test_sc = immoSV_test_sc.drop(['key_0','index','ocean_proximity'], axis=1)
immoSV_test_sc.rename({0: 'NEAR_OCEAN', 1: 'NEAR_BAY',2: 'ISLAND', 3: 'INLAND', 4: 'H_OCEAN'}, inplace=True, axis=1)
immoSV_test_sc

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,NEAR_OCEAN,NEAR_BAY,ISLAND,INLAND,H_OCEAN
0,0.216024,0.524468,0.901961,0.065100,0.120944,0.086998,0.118605,0.159025,229500.0,0.0,0.0,0.0,0.0,1.0
1,0.676471,0.164894,0.196078,0.187769,0.278218,0.107570,0.266162,0.160874,163000.0,0.0,1.0,0.0,0.0,0.0
2,0.243408,0.504255,0.156863,0.037956,0.058531,0.019956,0.056917,0.285320,234300.0,1.0,0.0,0.0,0.0,0.0
3,0.596349,0.198936,0.470588,0.081712,0.080733,0.049329,0.083073,0.330258,183000.0,1.0,0.0,0.0,0.0,0.0
4,0.215010,0.559574,1.000000,0.036175,0.034932,0.017069,0.034545,0.294541,314700.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4896,0.251521,0.514894,0.294118,0.033656,0.059773,0.030830,0.067774,0.221749,229800.0,1.0,0.0,0.0,0.0,0.0
4897,0.613590,0.160638,0.901961,0.035081,0.134762,0.080075,0.135549,0.066765,137500.0,1.0,0.0,0.0,0.0,0.0
4898,0.417850,0.564894,0.294118,0.007963,0.008850,0.003700,0.009541,0.135281,137500.0,0.0,1.0,0.0,0.0,0.0
4899,0.647059,0.146809,0.588235,0.049963,0.057289,0.025645,0.056588,0.269707,202000.0,1.0,0.0,0.0,0.0,0.0


In [219]:
immoSV_train_enc = immoSV_train_sc.copy()
immoSV_test_enc = immoSV_test_sc.copy()

### Definition X et y

In [220]:
# X = df.drop(["median_house_value"], axis=1)
# y = df[["median_house_value"]]

### Modelisation

In [337]:
X_train = immoSV_train_enc.drop(["median_house_value"], axis=1)
X_test = immoSV_test_enc.drop(["median_house_value"], axis=1)
y_train = immoSV_train_enc[["median_house_value"]]
y_test = immoSV_test_enc[["median_house_value"]]

In [338]:
model = KNeighborsRegressor()

In [339]:
# # 5-Fold Cross validate model
# cv_results = cross_validate(model, X, y, cv=5)
# # obtain the mean of scores
# cv_results['test_score'].mean()

In [340]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8033659719740243
Test score : 0.6969513293641078


In [341]:
y_pred = model.predict(X_test)
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [342]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.6899590322964562
MSE : 4082423078.4314575
RMSE : 63863.2384693946
MAE : 43361.0390380411


In [346]:
%%time
# Perform Permutation
permutation_score = permutation_importance(model, X_test, y_test,random_state=101,n_repeats=100) 
# Unstack results
importance_df = pd.DataFrame(np.vstack((X_test.columns,permutation_score.importances_mean)).T) 
importance_df.columns=['feature','feature importance']
# Order by importance
importance_df.sort_values(by="feature importance", ascending = False) 

Wall time: 7min 23s


,feature,feature importance
7,median_income,0.696361
1,latitude,0.189514
0,longitude,0.188491
2,housing_median_age,0.122081
4,total_bedrooms,0.0324433
5,population,0.0257734
6,households,0.019975
3,total_rooms,0.0131713
12,H_OCEAN,0.00833091
9,NEAR_BAY,0.00302384


In [345]:
vif_df = pd.DataFrame()
vif_df["vif_index"] = [vif(X_train.values, i) for i in range(X_train.shape[1])]
vif_df["features"] = X_train.columns
vif_df

,vif_index,features
0,17.989983,longitude
1,19.719951,latitude
2,1.328268,housing_median_age
3,13.065354,total_rooms
4,36.014650,total_bedrooms
5,6.102097,population
6,33.998210,households
7,1.782501,median_income
8,127.458373,NEAR_OCEAN
9,116.400382,NEAR_BAY


### Feature importance

In [227]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND"], axis=1)

In [228]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [229]:
# X_train.to_csv('./datasets/X_train.csv', index=False)
# X_test.to_csv('./datasets/X_test.csv', index=False)
# y_train.to_csv('./datasets/y_train.csv', index=False)
# y_test.to_csv('./datasets/y_test.csv', index=False)

In [230]:
# X_train = pd.read_csv('./datasets/X_train.csv')
# X_test = pd.read_csv('./datasets/X_test.csv')
# y_train = pd.read_csv('./datasets/y_train.csv')
# y_test = pd.read_csv('./datasets/y_test.csv')

In [231]:
model = KNeighborsRegressor()

In [232]:
# # 5-Fold Cross validate model
# cv_results = cross_validate(model, X, y, cv=5)
# # obtain the mean of scores
# cv_results['test_score'].mean()

In [233]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8030444440082032
Test score : 0.6968812610151582


In [234]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [235]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.6896264484170945
MSE : 4086810033.931586
RMSE : 63898.09826508343
MAE : 43377.4658154788


### GridSearchCV : n_neighbors=11

In [236]:
# n_neighbors=11

In [237]:
model = KNeighborsRegressor(n_neighbors=11)

In [238]:
# # 5-Fold Cross validate model
# cv_results = cross_validate(model, X, y, cv=5)
# # obtain the mean of scores
# cv_results['test_score'].mean()

In [239]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.7540358719329356
Test score : 0.696905023489227


In [240]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [241]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.6983930763471108
MSE : 3971646992.8319755
RMSE : 62998.779015791064
MAE : 43086.47567675001


# Essais

#### Supression latitude et longitude

In [242]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","latitude"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","latitude"], axis=1)

In [243]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [244]:
model = KNeighborsRegressor()

In [245]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.7924166172236172
Test score : 0.6778552658667103


In [246]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [247]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.67554322007612
MSE : 4271895155.6954155
RMSE : 65335.4125407123
MAE : 44696.072671622205


#### Supression latitude

In [248]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","longitude"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","longitude"], axis=1)

In [249]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [250]:
model = KNeighborsRegressor()

In [251]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.7796958241004134
Test score : 0.6625007877951024


In [252]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [253]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.6572790243949048
MSE : 4511877541.075635
RMSE : 67150.27719583767
MAE : 46121.970459116754


#### Supression longitude

In [254]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","latitude"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","latitude"], axis=1)

In [255]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [256]:
model = KNeighborsRegressor()

In [257]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.7924166172236172
Test score : 0.6778552658667103


In [258]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [259]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.67554322007612
MSE : 4271895155.6954155
RMSE : 65335.4125407123
MAE : 44696.072671622205


#### Supression housing_median_age

In [260]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","housing_median_age"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","housing_median_age"], axis=1)

In [261]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [262]:
model = KNeighborsRegressor()

In [263]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8294758690996982
Test score : 0.7299813558747059


In [264]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [265]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.7313596169786835
MSE : 3539225742.2002764
RMSE : 59465.773184455335
MAE : 39912.82728465238


#### Supression total_rooms

In [266]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","total_rooms"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","total_rooms"], axis=1)

In [267]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [268]:
model = KNeighborsRegressor()

In [269]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8040260094658285
Test score : 0.6979310897636216


In [270]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [271]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.6907797452883787
MSE : 4071599267.829901
RMSE : 63779.709476763484
MAE : 43326.901880192396


#### Supression total_bedrooms

In [272]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","total_bedrooms"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","total_bedrooms"], axis=1)

In [273]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [274]:
model = KNeighborsRegressor()

In [275]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8066846469199059
Test score : 0.6974535281774239


In [276]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [277]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.6935726020874231
MSE : 4034774373.785777
RMSE : 63495.066176688124
MAE : 43021.65133362483


#### Supression population

In [278]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","population"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","population"], axis=1)

In [279]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [280]:
model = KNeighborsRegressor()

In [281]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.7954892557291503
Test score : 0.684384361454441


In [282]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [283]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.6789949143742069
MSE : 4227057653.723687
RMSE : 64986.93503541362
MAE : 44182.51380848273


#### Supression households

In [284]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","households"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","households"], axis=1)

In [285]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [286]:
model = KNeighborsRegressor()

In [287]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8064520853317887
Test score : 0.7007299784642689


In [288]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [289]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.6939593760302784
MSE : 4029839967.706543
RMSE : 63458.18097511929
MAE : 43010.66214254481


#### Supression median_income

In [290]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","median_income"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","median_income"], axis=1)

In [291]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [292]:
model = KNeighborsRegressor()

In [293]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.7127863427533523
Test score : 0.5708194434535208


In [294]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [295]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.5392271632515314
MSE : 6067936652.791879
RMSE : 77881.58448881877
MAE : 54381.50845649322


#### Supression NEAR_OCEAN, NEAR_BAY, INLAND

In [296]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","NEAR_OCEAN", "NEAR_BAY", "INLAND"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","NEAR_OCEAN", "NEAR_BAY", "INLAND"], axis=1)

In [297]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [298]:
model = KNeighborsRegressor()

In [299]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8001353146355951
Test score : 0.6902239766789883


In [300]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [301]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.6879421326860511
MSE : 4109600640.978386
RMSE : 64088.81774129231
MAE : 43577.835347616965


#### Supression housing_median_age

In [302]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","housing_median_age"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","housing_median_age"], axis=1)

In [303]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [304]:
model = KNeighborsRegressor()

In [305]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8294758690996982
Test score : 0.7299813558747059


In [306]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [307]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.7313596169786835
MSE : 3539225742.2002764
RMSE : 59465.773184455335
MAE : 39912.82728465238


#### Supression housing_median_age, total_rooms, total_bedrooms, households

In [308]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households"], axis=1)

In [309]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [310]:
model = KNeighborsRegressor()

In [311]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8329222665446092
Test score : 0.7387178967620454


In [312]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [313]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.7399614081576986
MSE : 3424691328.6119356
RMSE : 58503.06736624155
MAE : 39473.568430257976


#### Supression housing_median_age, total_rooms, total_bedrooms, households et (n_neighbors=11)

In [314]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households"], axis=1)

In [315]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [316]:
model = KNeighborsRegressor(n_neighbors=11)

In [317]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.7926652667727929
Test score : 0.7447162100882985


In [318]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [319]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.7430868087135952
MSE : 3384423542.790679
RMSE : 58152.31762719669
MAE : 39498.82035218825


#### Supression housing_median_age, total_rooms, total_bedrooms, households,  NEAR_OCEAN, INLAND

In [320]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)

In [321]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [322]:
model = KNeighborsRegressor()

In [323]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.8315162189451739
Test score : 0.7324213818295806


In [324]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [325]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.7365323145577355
MSE : 3469819525.175237
RMSE : 58894.78070291308
MAE : 39777.36232619152


#### Supression housing_median_age, total_rooms, total_bedrooms, households,  NEAR_OCEAN, INLAND et (n_neighbors=11)

In [326]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)

In [327]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [328]:
model = KNeighborsRegressor(n_neighbors=11)

In [329]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.7908990320930849
Test score : 0.7374461118376059


In [330]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [331]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.7407571751864601
MSE : 3414878374.145632
RMSE : 58417.36647614559
MAE : 39650.958691417894


# GridSearch

In [374]:
param_grid = [
    {
        'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
        'weights' : ['uniform', 'distance'],
        'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
    }
]

In [375]:
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring=['r2','neg_root_mean_squared_error'],refit='neg_root_mean_squared_error', n_jobs=-1, verbose=True,return_train_score=True)

In [376]:
%%time
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 152 candidates, totalling 760 fits
Wall time: 7min 3s


GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid=[{'algorithm': ['auto', 'ball_tree', 'kd_tree',
                                        'brute'],
                          'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15, 16, 17, 18, 19, 20],
                          'weights': ['uniform', 'distance']}],
             refit='neg_root_mean_squared_error', return_train_score=True,
             scoring=['r2', 'neg_root_mean_squared_error'], verbose=True)

In [377]:
print(grid_search.best_params_, grid_search.best_score_)

{'algorithm': 'auto', 'n_neighbors': 12, 'weights': 'distance'} -57354.32817149929


#### Supression housing_median_age, total_rooms, total_bedrooms, households,  NEAR_OCEAN, INLAND et (n_neighbors=12)

In [351]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)

In [352]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [369]:
model = KNeighborsRegressor(n_neighbors=13)

In [370]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 0.7839078206375258
Test score : 0.7337894972792898


In [371]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [372]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.739580049541297
MSE : 3430474113.3468866
RMSE : 58549.73509938696
MAE : 39822.4646732367


#### Supression housing_median_age, total_rooms, total_bedrooms, households,  NEAR_OCEAN, INLAND et (n_neighbors=12)

In [351]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)

In [352]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [378]:
model = KNeighborsRegressor(algorithm='auto', n_neighbors= 12, weights='distance')

In [379]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 1.0
Test score : 0.7447216898677488


In [380]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [381]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.750121705939226
MSE : 3291651612.632971
RMSE : 57354.56326826895
MAE : 38733.39579968291


#### Supression housing_median_age, total_rooms, total_bedrooms, households,  NEAR_OCEAN, INLAND et (n_neighbors=9)

In [351]:
df_train = immoSV_train_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)
df_test = immoSV_test_enc.drop(["H_OCEAN","ISLAND","housing_median_age","total_rooms", "total_bedrooms", "households", "NEAR_OCEAN", "INLAND"], axis=1)

In [352]:
X_train = df_train.drop(["median_house_value"], axis=1)
X_test = df_test.drop(["median_house_value"], axis=1)
y_train = df_train[["median_house_value"]]
y_test = df_test[["median_house_value"]]

In [382]:
model = KNeighborsRegressor(algorithm='auto', n_neighbors= 9, weights='distance')

In [383]:
model.fit(X_train,y_train)
model.predict(X_test)
print(f"Train score : {model.score(X_train, y_train)}")
print(f"Test score : {model.score(X_test, y_test)}")

Train score : 1.0
Test score : 0.7458782339731238


In [384]:
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"RMSE : {np.sqrt(mean_squared_error(y_test, y_pred))}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

MSE : 4106551415.4368496
RMSE : 64082.379913958015
MAE : 43397.53331973067


In [385]:
print(f"R² : {cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='r2').mean()}")
print(f"MSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_squared_error').mean()}")
print(f"RMSE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_root_mean_squared_error').mean()}")
print(f"MAE : {-cross_val_score(model,X_train, y_train,cv=KFold(n_splits=5, shuffle=True, random_state=101),scoring='neg_mean_absolute_error').mean()}")

R² : 0.7502281039762715
MSE : 3290130256.725421
RMSE : 57343.704859058
MAE : 38748.22675703193
